In [1]:
import numpy as np
from steps import *
from tqdm import tqdm
from optimal import *
import matplotlib.pyplot as plt
import optuna
import pickle
import warnings
warnings.filterwarnings("error")


In [8]:
def plot_res(x_axis, results, x_label, y_label, title, path):
    plt.grid()
    plt.plot(x_axis, results.mean(axis=1))
    plt.fill_between(x_axis, results.mean(axis=1)+results.std(axis=1), results.mean(axis=1)-results.std(axis=1),
                     color='r', alpha=0.5)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.savefig(path+".png")
    plt.clf()

In [9]:
d = 20
r = 3
m = 10
T_r = 100
times = 10
d_sigma = 5
sigma_type = matrix_type.DIAG
s_type = matrix_type.IDENTITY

name = f"optuna_optimization_d={d}_r={r}_sigma={sigma_type}_s={s_type}_m={m}_Tr={T_r}_var={d_sigma}"
study = optuna.create_study(study_name=f'{name}',
                            storage=f'sqlite:///./../optuna/{name}.db ',
                            direction='minimize', load_if_exists=True)

[I 2023-05-06 16:45:24,213] Using an existing study with name 'optuna_optimization_d=20_r=3_sigma=matrix_type.DIAG_s=matrix_type.IDENTITY_m=10_Tr=100_var=5' instead of creating a new one.


In [9]:
# Sigma
sigma_max = 3
spaces = 1
samples = int((sigma_max-1) / spaces) + 1
d_sigmas = np.linspace(1, sigma_max, samples)

results = np.zeros([len(d_sigmas), times])

for idx, d_sigma in enumerate(d_sigmas):
    regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                 times=times, d_sigma=d_sigma, T_r=T_r,
                                                 **study.best_params)
    ratio = regret_list.min(axis=1) / regret_mix_list
    results[idx, :] = ratio

plot_res(x_axis=d_sigmas, results=results,
         x_label=r'$\sigma_0$', y_label='Accuracy Ratio',
         title=r'results accuracy ratio vs $\sigma_0$',
         path="results_accuracy_ratio_sigma0")
pickle.dump(results, open(f'acc-ratio_vs_sigma0.p', 'wb'))
pickle.dump(d_sigmas, open(f'sigma0-acc.p', 'wb'))

l_star=6 ; regret_mix=2.30326671793824 ; last_regret=3.1075351432946627
l_star=10 ; regret_mix=0.961732908100023 ; last_regret=1.2179073492896277
l_star=7 ; regret_mix=1.1580352415470174 ; last_regret=1.5436364806315457
l_star=10 ; regret_mix=1.2034353974109717 ; last_regret=1.5474432764505341
l_star=6 ; regret_mix=1.6459046514202456 ; last_regret=2.3495517192131623
l_star=8 ; regret_mix=2.28910557375703 ; last_regret=3.1890841734703286
l_star=7 ; regret_mix=3.259126284499907 ; last_regret=3.9580271894178916
l_star=9 ; regret_mix=3.089848452155753 ; last_regret=3.7361469114711348
l_star=6 ; regret_mix=3.405953069374457 ; last_regret=4.537606088059222
l_star=8 ; regret_mix=2.543333535282482 ; last_regret=3.4632582392990203
l_star=6 ; regret_mix=5.46906494357864 ; last_regret=6.587108346590821
l_star=7 ; regret_mix=5.603822980056652 ; last_regret=6.936993352306392
l_star=8 ; regret_mix=4.402297783722722 ; last_regret=5.598603335412095
l_star=8 ; regret_mix=4.152095704609059 ; last_regret

In [28]:
# r
r_max = 10
r_min = 1
spaces = 4
r_list = range(r_min, r_max, spaces)
m = 20

results = np.zeros([len(r_list), times])
times=5
for idx, r in enumerate(r_list):
    regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                 times=times, d_sigma=d_sigma, T_r=T_r,
                                                 **study.best_params)
    ratio = regret_list.min(axis=1) / regret_mix_list
    results[idx, :] = ratio

plot_res(x_axis=r_list, results=results,
         x_label='r', y_label='Accuracy Ratio',
         title='results accuracy ratio vs r',
         path="results_accuracy_ratio_r")
pickle.dump(results, open(f'acc-ratio_vs_r.p', 'wb'))
pickle.dump(r_list, open(f'r-acc.p', 'wb'))

l_star=2 ; regret_mix=10.440890768486565 ; last_regret=10.539386004815011
l_star=4 ; regret_mix=11.740001319372638 ; last_regret=12.082175005709738
l_star=3 ; regret_mix=6.491073673971406 ; last_regret=6.8395020680290175
l_star=3 ; regret_mix=4.202814974829063 ; last_regret=4.5980269371964795
l_star=6 ; regret_mix=4.431079235552793 ; last_regret=4.685845393039574
l_star=9 ; regret_mix=2.5743381160543484 ; last_regret=3.994021926664039
l_star=10 ; regret_mix=4.325792418741918 ; last_regret=5.888859452079774
l_star=11 ; regret_mix=2.6518408261490847 ; last_regret=3.5881729477066377
l_star=12 ; regret_mix=2.160991921486688 ; last_regret=3.0599508852852306
l_star=13 ; regret_mix=1.93622189261607 ; last_regret=2.54506310116334
l_star=16 ; regret_mix=1.2864618715132117 ; last_regret=2.1405686672878934
l_star=13 ; regret_mix=1.8488884775843237 ; last_regret=3.9165198140499884
l_star=15 ; regret_mix=0.7867438078805885 ; last_regret=1.2690876339985473
l_star=14 ; regret_mix=1.1908788755889446 ;

In [ ]:
# d
d_max = 20
d_min = 5
spaces = 1
d_list = range(d_min, d_max, spaces)

results = np.zeros([len(d_list), times])
times=5
for idx, d in enumerate(d_list):
    regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                 times=times, d_sigma=d_sigma, T_r=T_r,
                                                 **study.best_params)
    ratio = regret_list.min(axis=1) / regret_mix_list
    results[idx, :] = ratio

plot_res(x_axis=d_list, results=results,
         x_label='d', y_label='Accuracy Ratio',
         title='results accuracy ratio vs d',
         path="results_accuracy_ratio_d")
pickle.dump(results, open(f'acc-ratio_vs_d.p', 'wb'))
pickle.dump(r_list, open(f'd-acc.p', 'wb'))

In [10]:
# Regrets
regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                 times=times, d_sigma=d_sigma, T_r=T_r,
                                                 **study.best_params)

plot_res(x_axis=range(m+2), results=regret_list.T,
         x_label='#Iteration', y_label='Regret',
         title='Regret vs #Iteration',
         path="Regret_vs_Iteration")

l_star=7 ; regret_mix=7.566881593609462 ; last_regret=9.082926827910175
l_star=7 ; regret_mix=7.568180103786526 ; last_regret=8.070292900695208
l_star=7 ; regret_mix=6.381224333426599 ; last_regret=6.852635935873047
l_star=7 ; regret_mix=8.99067348969484 ; last_regret=9.756318919967685
l_star=5 ; regret_mix=7.958422879842154 ; last_regret=8.181594194431858
l_star=7 ; regret_mix=10.140460202290345 ; last_regret=11.087857762367465
l_star=7 ; regret_mix=10.497193920589144 ; last_regret=10.896601999361323
l_star=7 ; regret_mix=6.304138394419756 ; last_regret=6.746970842836093
l_star=8 ; regret_mix=6.866200414356532 ; last_regret=7.825228371518902
l_star=8 ; regret_mix=3.4390694572894733 ; last_regret=4.164546732070919


array([[2.4806076 , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.73220407, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.72424764, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.10078782, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.26833035]])